In [ ]:
import sys, os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

from src import *
from src.value import *
import numpy as np
import time, math, json
import plotly.express as px
import pandas as pd

# Training

In [ ]:
# load images
train_img_path = "../data/train_img.idx"
train_label_path = "../data/train_label.idx"
test_img_path = "../data/test_img.idx"
test_label_path = "../data/test_label.idx"

train_img, train_label, test_img, test_label = binary_parse_mnist_data(
    train_img_path, train_label_path, test_img_path, test_label_path, 0, 1
)

# get equally distributed images and labels
train_img, train_label = get_number_of_samples(train_img, train_label, 90)
test_img, test_label = get_number_of_samples(test_img, test_label, 90)

# initialize MLP
nin = 100
n_hidden = [10, 10]
nout = 1
mlp = MLP(nin, n_hidden, nout)

In [ ]:
np.random.seed(0xDEADBEEF)
# Hyperparameter
lr = 1e-3
epochs = 10
batch_size = 30
num_img = train_img.shape[0]
num_batches = math.ceil(num_img / batch_size)

# Plot Parameter
losses_test = []
losses_train = []
accuracies_test = []
accuracies_train = []
times = []

for e in range(epochs):
    idx = np.random.permutation(np.arange(num_img))
    # inplace for better cache usage
    train_img = train_img[idx]
    train_label = train_label[idx]
    test_img = test_img[idx]
    test_label = test_label[idx]

    # Genauigkeit & Loss berechnen 1 pro Epoche für Plot
    print("calculating accuracies and losses...")
    train_loss, train_accuracy = mlp.epoch_loss_and_accuracy(
        images=train_img, labels=train_label
    )
    test_loss, test_accuracy = mlp.epoch_loss_and_accuracy(
        images=test_img, labels=test_label
    )
    losses_train.append(train_loss.value)
    losses_test.append(test_loss.value)
    accuracies_train.append(train_accuracy)
    accuracies_test.append(test_accuracy)
    print("...done")

    # Epochendauer ausgeben
    start_time = time.process_time()

    for b in range(num_batches):
        batch_start_time = time.process_time()
        start_sample = b * batch_size
        end_sample = min((b + 1) * batch_size, num_img)
        x = train_img[start_sample:end_sample]
        y_gt = train_label[start_sample:end_sample]

        # zero grad
        for p in mlp.parameters():
            p.grad = 0.0

        # forward pass
        y_pred = [mlp(img) for img in x]

        # backward pass
        outputs = [
            Value.cross_entropy_loss(ypred, ygt) for ypred, ygt in zip(y_pred, y_gt)
        ]
        loss = sum(outputs) / len(outputs)
        loss.backward()

        # optimization
        for p in mlp.parameters():
            p.value -= lr * p.grad
        batch_end_time = time.process_time()
        print(
            f"Batchdauer: {batch_end_time-batch_start_time}, Size: {end_sample-start_sample}"
        )

    end_time = time.process_time()
    times.append(end_time - start_time)
    print(f"Epoche {e+1}: {times[e]} s")

time_df = pd.DataFrame(
    {
        "epochs": range(epochs),
        "times": times,
    }
)

loss_df = pd.DataFrame(
    {
        "epochs": range(epochs),
        "train_loss": losses_train,
        "test_loss": losses_test,
    }
)

acc_df = pd.DataFrame(
    {
        "epochs": range(epochs),
        "train_acc": accuracies_train,
        "test_acc": accuracies_test,
    }
)

loss_df = loss_df.melt(
    id_vars="epochs", value_vars=["train_loss", "test_loss"], value_name="loss"
)
acc_df = acc_df.melt(
    id_vars="epochs", value_vars=["train_acc", "test_acc"], value_name="accuracy"
)

In [ ]:
px.line(time_df, x="epochs", y="times")

In [ ]:
fig = px.line(loss_df, x="epochs", y="loss", color="variable")
fig.show()

In [ ]:
fig = px.line(acc_df, x="epochs", y="accuracy", color="variable")
fig.show()

# Long Loop

In [ ]:
train_img, train_label, test_img, test_label = binary_parse_mnist_data(
    train_img_path, train_label_path, test_img_path, test_label_path, 0, 1
)

train_img, train_label = get_number_of_samples(train_img, train_label, 900)
test_img, test_label = get_number_of_samples(test_img, test_label, 900)
# initialize MLP
nin = 100
n_hidden = [10, 10]
nout = 1
mlp2 = MLP(nin, n_hidden, nout)

# Hyperparameter
lr = 1e-3
epochs = 10
batch_size = 30
num_img = train_img.shape[0]
num_batches = math.ceil(num_img / batch_size)

count_epochs = 0
timer = 0
datas = []

# langer Durchlauf -> Plot Werte periodisch in Datei
while True:
    idx = np.random.permutation(np.arange(num_img))
    # inplace for better cache usage
    train_img = train_img[idx]
    train_label = train_label[idx]

    test_img = test_img[idx]
    test_label = test_label[idx]

    # Genauigkeit & Loss berechnen 1 pro 100 Epochen für Plot

    train_loss, train_accuracy = mlp2.epoch_loss_and_accuracy(
        images=train_img, labels=train_label
    )
    test_loss, test_accuracy = mlp2.epoch_loss_and_accuracy(
        images=test_img, labels=test_label
    )
    # TODO Accuracy, Loss und Timer in Datei Schreiben
    data = {
        "epoch": count_epochs,
        "time": timer,
        "train_loss": train_loss.value,
        "train_acc": train_accuracy,
        "test_loss": test_loss.value,
        "test_acc": test_accuracy,
    }
    datas.append(data)
    with open("../data/log_training.json", "w") as out:
        json.dump(datas, out, indent=4)
    # Epochendauer ausgeben
    start_time = time.process_time()

    for b in range(num_batches):
        start_sample = b * batch_size
        end_sample = min((b + 1) * batch_size, num_img)
        x = train_img[start_sample:end_sample]
        y_gt = train_label[start_sample:end_sample]

        # zero grad
        for p in mlp2.parameters():
            p.grad = 0.0

        # forward pass
        y_pred = [mlp2(img) for img in x]

        # backward pass
        outputs = [
            Value.cross_entropy_loss(ypred, ygt) for ypred, ygt in zip(y_pred, y_gt)
        ]
        loss = sum(outputs) / len(outputs)
        loss.backward()

        # optimization
        for p in mlp2.parameters():
            p.value -= lr * p.grad

    end_time = time.process_time()
    timer = end_time - start_time
    count_epochs += 1